In [1]:
import os

In [2]:
%pwd

'd:\\Angus_Issues\\NorthCarolina_CameroonChapter_AngusIssues\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Angus_Issues\\NorthCarolina_CameroonChapter_AngusIssues'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ETSModel
        schema =  self.schema.TARGET_COLUMN
        target_column = list(self.schema['TARGET_COLUMN'].keys())[0]


        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = target_column
           
        )

        return model_evaluation_config


In [8]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        #mae = mean_absolute_error(actual, pred)
        #r2 = r2_score(actual, pred)
        #return rmse, mae, r2
        return rmse


    def save_results(self):

        data = pd.read_csv(self.config.data_path)
        data.drop('Unnamed: 0', axis=1, inplace=True)
        data['Date'] = pd.to_datetime(data['Date']) # Convert 'Date' column to datetime format
        data = data.set_index('Date')

        # Initialize forecast results DataFrame
        forecast_results = pd.DataFrame(columns=['Date', 'Cow', 'Predicted Body Weight (kg)'])

        # Define the date to split the data (adjust the date as needed)
        split_date = pd.to_datetime('2022-02-15')

        # Extract unique cow IDs from the DataFrame
        unique_cows = data['Cow'].unique()
        

        forecast_results = pd.read_csv(self.config.test_data_path) #importing forecast.csv
        

        # Extract unique cow IDs from the DataFrame
        unique_cows = data['Cow'].unique()
        actual_values = []
        predicted_values = []

        for cow in unique_cows:
            cow_data = data[data['Cow'] == cow]

            # Split the data into train and test based on the split_date
            cow_test_data = cow_data[cow_data.index >= split_date]

            # Extract actual values for the test set
            actual_values.extend(cow_test_data['Body Weight (kg)'].values)

            # Extract predicted values for the test set from forecast_results
            cow_predictions = forecast_results[forecast_results['Cow'] == cow]['Predicted Body Weight (kg)'].values
            predicted_values.extend(cow_predictions)

        # Calculate RMSE
        rmse = np.sqrt(mean_squared_error(actual_values, predicted_values))
        print(f"RMSE for ETS Model: {rmse}")


        
        # Saving metrics as local
        #scores = {"rmse": rmse, "mae": mae, "r2": r2}
        scores = {"rmse": rmse}
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-08-18 15:50:15,973: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-18 15:50:15,994: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-18 15:50:16,008: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-18 15:50:16,011: INFO: common: created directory at: artifacts]
[2024-08-18 15:50:16,012: INFO: common: created directory at: artifacts/model_evaluation]
RMSE for ETS Model: 103.4605447353742
[2024-08-18 15:50:16,172: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
